<a href="https://colab.research.google.com/github/josvaldes/trabajoGradoMCD/blob/feature%2Fnotebook1_colombiatic/scrapingColombiaTicFinal3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
#  Notebook 1 - ColombiaTIC: Scraping + Carga Incremental a DuckDB
#  Versión optimizada: conexión segura, control_cargue, y logs persistentes
# ============================================================

# ========== 0. Preparación e instalaciones (auto) ==========
import sys, subprocess, importlib

def pip_install(pkgs):
    for p in pkgs:
        try:
            importlib.import_module(p if p != 'requests_html' else 'requests_html')
        except ImportError:
            print(f"📦 Instalando {p}...")
            subprocess.run([sys.executable, "-m", "pip", "install", p, "-q"], check=True)

# Paquetes requeridos
pip_install(["duckdb", "pandas", "openpyxl", "tqdm", "requests", "requests_html", "lxml_html_clean", "nest_asyncio"])

# ========== 1. Imports y paths ==========
import os, re, shutil, time, logging
from datetime import datetime
import duckdb
import pandas as pd
import requests
from tqdm import tqdm
import nest_asyncio
nest_asyncio.apply()

from requests_html import AsyncHTMLSession
from google.colab import drive

# Rutas base (ajusta si lo necesitas)
RUTA_EXCEL_DEST   = "/content/gdrive/MyDrive/trabajoGrado/reporte_colombiatic"
RUTA_EXCEL_TEMP   = "/content/gdrive/MyDrive/trabajoGrado/temp_colombiatic"
RUTA_DB_DRIVE_DIR = "/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos"
RUTA_DB_DRIVE     = os.path.join(RUTA_DB_DRIVE_DIR, "colombiatic.duckdb")
RUTA_DB_LOCAL     = "/content/colombiatic_temp.duckdb"     # base temporal local para evitar IO de Drive
RUTA_LOG          = os.path.join(RUTA_DB_DRIVE_DIR, "colombiatic_etl.log")  # LOG persistente junto a la base

os.makedirs(RUTA_EXCEL_DEST, exist_ok=True)
os.makedirs(RUTA_EXCEL_TEMP, exist_ok=True)
os.makedirs(RUTA_DB_DRIVE_DIR, exist_ok=True)

# ============================================================
# 🔗 Montaje robusto de Google Drive
# ============================================================
from google.colab import drive
import os
import shutil

mount_path = "/content/gdrive"

try:
    # Si ya existe y contiene archivos, lo desmontamos primero
    if os.path.exists(mount_path) and os.listdir(mount_path):
        print("⚙️  Desmontando Drive previo...")
        drive.flush_and_unmount()
        shutil.rmtree(mount_path, ignore_errors=True)

    # Montar Drive limpio
    drive.mount(mount_path, force_remount=True)
    print("✅ Google Drive montado correctamente en:", mount_path)
except Exception as e:
    print(f"⚠️ Error al montar Google Drive: {e}")



# ========== 2. Logger persistente ==========
logger = logging.getLogger("colombiatic_etl")
logger.setLevel(logging.INFO)
# Evitar handlers duplicados en re-ejecuciones
if not logger.handlers:
    fh = logging.FileHandler(RUTA_LOG, encoding="utf-8")
    fh.setLevel(logging.INFO)
    fmt = logging.Formatter("%(asctime)s | %(levelname)s | %(message)s")
    fh.setFormatter(fmt)
    logger.addHandler(fh)

def log_info(msg):
    print(msg)
    logger.info(msg)

def log_warn(msg):
    print(msg)
    logger.warning(msg)

def log_error(msg):
    print(msg)
    logger.error(msg)

log_info("===== INICIO EJECUCIÓN NOTEBOOK 1 (Scraping + ETL) =====")
log_info(f"Carpeta destino Excel: {RUTA_EXCEL_DEST}")
log_info(f"Carpeta temporal Excel: {RUTA_EXCEL_TEMP}")
log_info(f"Base en Drive: {RUTA_DB_DRIVE}")
log_info(f"Log persistente: {RUTA_LOG}")

# ========== 3. Utilidades (nombres y conexión) ==========
def normalizar_nombre(s: str, max_len: int = 60) -> str:
    s = s.lower()
    s = re.sub(r"[^a-z0-9_]", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s[:max_len] if max_len else s

def conectar_duckdb_seguro(path_db: str):
    """Conecta o crea base. Si existe en Drive, copia a local y conecta local."""
    # Si existe base en Drive, hacer copia local para trabajar rápido/sin locks
    if os.path.exists(RUTA_DB_DRIVE):
        try:
            if os.path.exists(RUTA_DB_LOCAL):
                os.remove(RUTA_DB_LOCAL)
            shutil.copy2(RUTA_DB_DRIVE, RUTA_DB_LOCAL)
            log_info(f"✅ Copia local creada desde Drive: {RUTA_DB_DRIVE} → {RUTA_DB_LOCAL}")
        except Exception as e:
            log_warn(f"⚠️ No se pudo copiar base desde Drive. Se creará una nueva local. Detalle: {e}")
    # Conectar a la copia local (o nueva si no existía)
    con = duckdb.connect(RUTA_DB_LOCAL)
    log_info(f"💾 Conexión establecida con base local: {RUTA_DB_LOCAL}")
    # Tabla de control (si no existe)
    con.execute("""
    CREATE TABLE IF NOT EXISTS control_cargue (
        archivo TEXT,
        hojas_cargadas INTEGER,
        filas_totales INTEGER,
        fecha_cargue TIMESTAMP,
        estado TEXT
    )
    """)
    log_info("📊 Tabla 'control_cargue' lista.")
    return con

def sincronizar_local_a_drive():
    """Cierra conexión si está abierta y copia local → Drive."""
    try:
        con.close()
    except:
        pass
    try:
        if os.path.exists(RUTA_DB_LOCAL):
            shutil.copy2(RUTA_DB_LOCAL, RUTA_DB_DRIVE)
            log_info(f"📦 Base actualizada copiada de nuevo al Drive: {RUTA_DB_LOCAL} → {RUTA_DB_DRIVE}")
    except Exception as e:
        log_error(f"❌ Error copiando base a Drive: {e}")

# ========== 4. Scraping (encuentra y descarga solo Excel; filtra TIC) ==========
URL_PORTAL = "https://colombiatic.mintic.gov.co/679/w3-channel.html"

async def obtener_urls_excel():
    """Rastrea artículos del canal y devuelve URLs .xlsx únicas."""
    session = AsyncHTMLSession()
    try:
        r = await session.get(URL_PORTAL)
        await r.html.arender(timeout=60, sleep=2)
        # enlances a artículos del canal
        links = r.html.absolute_links
        urls_articulos = [u for u in links if "/w3-article-" in u or "/articles-" in u]
        urls_articulos = list(dict.fromkeys(urls_articulos))  # únicos
        log_info(f"📰 Artículos detectados: {len(urls_articulos)}")

        urls_xlsx = set()
        for u in tqdm(urls_articulos, desc="🔎 Explorando artículos"):
            try:
                a = await session.get(u)
                await a.html.arender(timeout=60, sleep=1)
                for l in a.html.absolute_links:
                    if l.lower().endswith(".xlsx") or "archivo_xls.xlsx" in l.lower():
                        urls_xlsx.add(l)
            except Exception as e:
                log_warn(f"⚠️ Error explorando {u}: {e}")

        return list(urls_xlsx)
    finally:
        await session.close()

def descargar_excel(url: str, destino_dir: str) -> str | None:
    """Descarga un Excel y lo nombra a partir del slug del artículo o filename; retorna ruta destino."""
    try:
        resp = requests.get(url, timeout=60)
        if resp.status_code != 200:
            log_warn(f"⚠️ No se pudo descargar {url} (status {resp.status_code})")
            return None
        # nombre base por defecto
        base = url.split("/")[-1].replace(".xlsx", "")
        base = normalizar_nombre(base, 80)

        # guardar en temp
        nombre = f"{base}.xlsx"
        ruta = os.path.join(destino_dir, nombre)
        with open(ruta, "wb") as f:
            f.write(resp.content)
        return ruta
    except Exception as e:
        log_warn(f"⚠️ Error descargando {url}: {e}")
        return None

# Lanzar scraping
log_info("🌐 Iniciando scraping de boletines...")
try:
    # Ejecutar asincrónicamente en Colab
    import asyncio
    urls_xlsx = asyncio.get_event_loop().run_until_complete(obtener_urls_excel())
except RuntimeError:
    # Loop ya corriendo: usar nest_asyncio y crear uno nuevo
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    urls_xlsx = loop.run_until_complete(obtener_urls_excel())

log_info(f"📊 Archivos Excel detectados (únicos): {len(urls_xlsx)}")

# Descarga a carpeta temporal
descargados = []
for u in tqdm(urls_xlsx, desc="⬇️ Descargando archivos Excel"):
    ruta_tmp = descargar_excel(u, RUTA_EXCEL_TEMP)
    if ruta_tmp:
        descargados.append(ruta_tmp)

log_info(f"📦 Archivos descargados en temp: {len(descargados)}")

# Detectar TIC y mover a carpeta final. Borrar los no TIC
movidos = []
eliminados = []
for ruta in descargados:
    nombre = os.path.basename(ruta)
    # Heurística: debe contener "tic" en el nombre (en muchos casos funciona),
    # y/o es el archivo principal que el portal publica (ya trae "sector_tic" o similar).
    if "tic" in nombre.lower():
        destino = os.path.join(RUTA_EXCEL_DEST, nombre)
        if os.path.exists(destino):
            os.remove(destino)
        shutil.move(ruta, destino)
        movidos.append(destino)
        log_info(f"📦 Movido a carpeta destino: {os.path.basename(destino)}")
    else:
        os.remove(ruta)
        eliminados.append(nombre)
        log_info(f"🧹 Eliminado archivo no relevante: {nombre}")

# Limpieza final de temp (si quedó algo)
try:
    for f in os.listdir(RUTA_EXCEL_TEMP):
        try:
            os.remove(os.path.join(RUTA_EXCEL_TEMP, f))
        except:
            pass
except:
    pass

log_info("🎯 Scraping finalizado.")

# ========== 5. Conexión a DuckDB (local) ==========
# Importante: conectar después del scraping
con = conectar_duckdb_seguro(RUTA_DB_DRIVE)

# ========== 6. Carga incremental de Excel TIC ==========
def archivos_tic_actuales():
    return [f for f in os.listdir(RUTA_EXCEL_DEST) if f.lower().endswith((".xlsx", ".xls"))]

# Obtén los ya cargados desde control_cargue
try:
    ya_cargados = set(
        con.execute("SELECT DISTINCT archivo FROM control_cargue").fetchdf()["archivo"].tolist()
    )
except Exception:
    ya_cargados = set()

archivos = archivos_tic_actuales()
pendientes = [f for f in archivos if f not in ya_cargados]

print("\n📂 Archivos disponibles actualmente en carpeta destino:")
for f in archivos:
    print(f"   • {f}")

if not pendientes:
    log_info("⏭️ No hay archivos nuevos para cargar. La base está actualizada.")
else:
    log_info(f"🆕 Archivos TIC nuevos detectados: {len(pendientes)}")

for nombre_archivo in pendientes:
    ruta_archivo = os.path.join(RUTA_EXCEL_DEST, nombre_archivo)
    estado = "OK"
    total_filas = 0
    hojas_cargadas = 0
    log_info(f"\n📘 Procesando: {nombre_archivo}")

    try:
        xls = pd.ExcelFile(ruta_archivo)
        hojas = xls.sheet_names
        # proceso hoja a hoja
        for hoja in hojas:
            try:
                # Detectar encabezado: primeras 10 filas probando
                df_valid = None
                for fila_enc in range(0, 10):
                    df_try = pd.read_excel(ruta_archivo, sheet_name=hoja, header=fila_enc, engine="openpyxl")
                    if df_try.columns.notna().sum() > 2:
                        df_valid = df_try
                        break
                if df_valid is None or df_valid.empty:
                    continue

                df_valid = df_valid.astype(str)

                # nombre de tabla
                base = normalizar_nombre(os.path.splitext(nombre_archivo)[0], 48)
                hoja_n = normalizar_nombre(hoja, 10)  # corto para no pasarse de 63
                nombre_tabla = f"{base}_{hoja_n}"

                # Evitar duplicados de tabla
                tablas_existentes = con.execute("SHOW TABLES").fetchdf()["name"].tolist()
                if nombre_tabla in tablas_existentes:
                    log_info(f"   ⏭️ La tabla '{nombre_tabla}' ya existe, se omite.")
                else:
                    con.register("tmp_df", df_valid)
                    con.execute(f"CREATE TABLE '{nombre_tabla}' AS SELECT * FROM tmp_df")
                    con.unregister("tmp_df")
                    hojas_cargadas += 1
                    total_filas += len(df_valid)
                    log_info(f"   ✅ Hoja '{hoja}' cargada como '{nombre_tabla}' ({len(df_valid)} filas)")
            except Exception as e_hoja:
                estado = f"ERROR_HOJA:{hoja}:{str(e_hoja)[:150]}"
                log_warn(f"⚠️ {estado}")

    except Exception as e_file:
        estado = f"ERROR_ARCHIVO:{str(e_file)[:150]}"
        log_warn(f"⚠️ {estado}")

    # Registrar cargue a nivel archivo (incremental)
    try:
        con.execute("""
            INSERT INTO control_cargue (archivo, hojas_cargadas, filas_totales, fecha_cargue, estado)
            VALUES (?, ?, ?, ?, ?)
        """, [nombre_archivo, hojas_cargadas, total_filas, datetime.now(), estado])
        log_info(f"📝 Registrado en control_cargue: {nombre_archivo} | hojas={hojas_cargadas} | filas={total_filas} | estado={estado}")
    except Exception as e_ins:
        log_warn(f"⚠️ Error insertando en control_cargue: {e_ins}")

# Sincronizar base local a Drive y cerrar
sincronizar_local_a_drive()

# ========== 7. Revisar la base (resumen final) ==========
# Para evitar el error de conexión cerrada, abrimos una conexión **ligera** a la base en Drive solo para consulta
try:
    con_check = duckdb.connect(RUTA_DB_DRIVE, read_only=True)
    tablas = con_check.execute("SHOW TABLES").fetchdf()
    print("\n📂 Tablas en la base:")
    display(tablas)

    control = con_check.execute("""
        SELECT archivo, hojas_cargadas, filas_totales, fecha_cargue, estado
        FROM control_cargue
        ORDER BY fecha_cargue DESC
        LIMIT 20
    """).fetchdf()
    print("\n📋 Últimos registros en control_cargue:")
    display(control)

    total_control = con_check.execute("SELECT COUNT(*) AS total FROM control_cargue").fetchdf()
    print("\n🧮 Total de registros en control_cargue:")
    display(total_control)

    # tamaño del archivo de base
    tam_mb = os.path.getsize(RUTA_DB_DRIVE) / (1024*1024)
    print(f"\n💾 Tamaño actual de la base: {tam_mb:.2f} MB")
    con_check.close()
except Exception as e:
    log_warn(f"⚠️ No se pudo consultar la base final: {e}")

log_info("===== FIN EJECUCIÓN NOTEBOOK 1 =====")
print("\n✅ Proceso completado.")
print(f"🗂 Log del ETL: {RUTA_LOG}")
print(f"🗄️ Base DuckDB: {RUTA_DB_DRIVE}")
print(f"📁 Carpeta Excel (solo TIC): {RUTA_EXCEL_DEST}")


⚙️  Desmontando Drive previo...
Drive not mounted, so nothing to flush and unmount.
Mounted at /content/gdrive
✅ Google Drive montado correctamente en: /content/gdrive


INFO:colombiatic_etl:===== INICIO EJECUCIÓN NOTEBOOK 1 (Scraping + ETL) =====
INFO:colombiatic_etl:Carpeta destino Excel: /content/gdrive/MyDrive/trabajoGrado/reporte_colombiatic
INFO:colombiatic_etl:Carpeta temporal Excel: /content/gdrive/MyDrive/trabajoGrado/temp_colombiatic
INFO:colombiatic_etl:Base en Drive: /content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb
INFO:colombiatic_etl:Log persistente: /content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic_etl.log
INFO:colombiatic_etl:🌐 Iniciando scraping de boletines...


===== INICIO EJECUCIÓN NOTEBOOK 1 (Scraping + ETL) =====
Carpeta destino Excel: /content/gdrive/MyDrive/trabajoGrado/reporte_colombiatic
Carpeta temporal Excel: /content/gdrive/MyDrive/trabajoGrado/temp_colombiatic
Base en Drive: /content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb
Log persistente: /content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic_etl.log
🌐 Iniciando scraping de boletines...


[INFO] Starting Chromium download.
INFO:pyppeteer.chromium_downloader:Starting Chromium download.
100%|██████████| 183M/183M [00:00<00:00, 212Mb/s]
[INFO] Beginning extraction
INFO:pyppeteer.chromium_downloader:Beginning extraction
[INFO] Chromium extracted to: /root/.local/share/pyppeteer/local-chromium/1181205
INFO:pyppeteer.chromium_downloader:Chromium extracted to: /root/.local/share/pyppeteer/local-chromium/1181205
INFO:colombiatic_etl:📰 Artículos detectados: 6


📰 Artículos detectados: 6


...

,archivo,hojas_cargadas,filas_totales,fecha_cargue,estado
0,articles_404030_archivo_xls.xlsx,21,2363737,2025-10-23 18:32:13.245150,"ERROR_HOJA:18:Passed header=[4], len of 1, but..."
1,articles_407454_archivo_xls.xlsx,3,864401,2025-10-23 18:26:17.344438,"ERROR_HOJA:3:Passed header=[4], len of 1, but ..."
2,articles_406102_archivo_xls.xlsx,3,11398,2025-10-23 18:23:45.249435,OK
3,reporte_tic_tercer_trimestre_2024.xlsx,22,2364691,2025-10-23 02:24:33.008973,OK
4,reporte_tic_cuarto_trimestre_2024.xlsx,23,2365465,2025-10-23 02:18:42.719431,OK
5,reporte_tic_segundo_trimestre_2024.xlsx,23,2269671,2025-10-23 02:12:54.000494,OK
6,boletn_trimestral_del_sector_tic_cifras_primer...,23,2363774,2025-10-23 02:07:11.672925,OK
7,boletn_trimestral_del_sector_postal_cifras_seg...,3,11398,2025-10-23 01:57:11.487435,OK
8,boletn_trimestral_del_sector_de_tv_por_suscrip...,5,864438,2025-10-23 01:57:10.044003,OK
9,reporte_tic_tercer_trimestre_2024.xlsx,22,2364691,2025-10-23 01:54:42.788831,OK



🧮 Total de registros en control_cargue:


,total
0,19


INFO:colombiatic_etl:===== FIN EJECUCIÓN NOTEBOOK 1 =====



💾 Tamaño actual de la base: 154.51 MB
===== FIN EJECUCIÓN NOTEBOOK 1 =====

✅ Proceso completado.
🗂 Log del ETL: /content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic_etl.log
🗄️ Base DuckDB: /content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb
📁 Carpeta Excel (solo TIC): /content/gdrive/MyDrive/trabajoGrado/reporte_colombiatic


In [ ]:
import duckdb, pandas as pd

# Conexión (solo lectura)
db_path = "/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb"
con = duckdb.connect(db_path, read_only=True)

# Listar todas las tablas que contengan "_4_1"
tablas_41 = con.execute("""
    SELECT table_name AS name
    FROM duckdb_tables()
    WHERE table_name LIKE '%4_1%'
    ORDER BY table_name
""").fetchdf()

print("📋 Tablas 4_1 detectadas:")
display(tablas_41)



📋 Tablas 4_1 detectadas:


,name
0,articles_404030_archivo_xls_4_1
1,reporte_tic_cuarto_trimestre_2024_1
2,reporte_tic_cuarto_trimestre_2024_10
3,reporte_tic_cuarto_trimestre_2024_11
4,reporte_tic_cuarto_trimestre_2024_12
5,reporte_tic_cuarto_trimestre_2024_13
6,reporte_tic_cuarto_trimestre_2024_14
7,reporte_tic_cuarto_trimestre_2024_15
8,reporte_tic_cuarto_trimestre_2024_16
9,reporte_tic_cuarto_trimestre_2024_17


Bloque de código final para consolidar automáticamente

In [ ]:
import duckdb, pandas as pd

# --- Asegurar conexión limpia ---
try:
    con.close()
except:
    pass

db_path = "/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb"
con = duckdb.connect(db_path)
print("✅ Conectado a la base correctamente.\n")

# --- Buscar todas las tablas que terminen en _4_1 ---
tablas_41 = con.execute("""
    SELECT table_name AS name
    FROM duckdb_tables()
    WHERE table_name LIKE '%4_1'
    ORDER BY table_name
""").fetchdf()

if tablas_41.empty:
    print("⚠️ No hay tablas 4.1 disponibles.")
else:
    print(f"📋 Tablas detectadas para consolidar ({len(tablas_41)}):")
    display(tablas_41)

    # --- Obtener todas las columnas únicas entre las tablas 4.1 ---
    columnas_union = set()
    columnas_por_tabla = {}

    for t in tablas_41["name"]:
        cols = con.execute(f"PRAGMA table_info('{t}')").fetchdf()["name"].tolist()
        columnas_union.update(cols)
        columnas_por_tabla[t] = cols

    columnas_union = sorted(list(columnas_union))
    print(f"🧱 Total columnas únicas detectadas: {len(columnas_union)}\n")

    # --- Crear sentencias SELECT alineadas ---
    selects = []
    for t, cols in columnas_por_tabla.items():
        # agregar NULL para columnas faltantes
        select_cols = [f"'{t}' AS origen"]
        for col in columnas_union:
            if col in cols:
                select_cols.append(f'"{col}"')
            else:
                select_cols.append(f"NULL AS \"{col}\"")
        selects.append(f"SELECT {', '.join(select_cols)} FROM '{t}'")

    union_query = " UNION ALL ".join(selects)

    # --- Crear tabla consolidada ---
    con.execute(f"""
        CREATE OR REPLACE TABLE consolidado_tic_4_1 AS {union_query}
    """)
    print("✅ Tabla consolidada creada: consolidado_tic_4_1")

    # --- Mostrar resumen de consolidación ---
    resumen = con.execute("""
        SELECT origen, COUNT(*) AS registros
        FROM consolidado_tic_4_1
        GROUP BY origen
        ORDER BY origen
    """).fetchdf()

    print("\n📊 Registros por tabla origen:")
    display(resumen)

    total = con.execute("SELECT COUNT(*) FROM consolidado_tic_4_1").fetchone()[0]
    print(f"🧮 Total de registros consolidados: {total:,}")

con.close()
print("\n🔚 Conexión cerrada correctamente.")


✅ Conectado a la base correctamente.

📋 Tablas detectadas para consolidar (7):


,name
0,articles_404030_archivo_xls_4_1
1,reporte_tic_cuarto_trimestre_2024_1
2,reporte_tic_cuarto_trimestre_2024_4_1
3,reporte_tic_segundo_trimestre_2024_1
4,reporte_tic_segundo_trimestre_2024_4_1
5,reporte_tic_tercer_trimestre_2024_1
6,reporte_tic_tercer_trimestre_2024_4_1


🧱 Total columnas únicas detectadas: 12



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Tabla consolidada creada: consolidado_tic_4_1

📊 Registros por tabla origen:


,origen,registros
0,articles_404030_archivo_xls_4_1,385911
1,reporte_tic_cuarto_trimestre_2024_1,11698
2,reporte_tic_cuarto_trimestre_2024_4_1,571379
3,reporte_tic_segundo_trimestre_2024_1,10657
4,reporte_tic_segundo_trimestre_2024_4_1,104218
5,reporte_tic_tercer_trimestre_2024_1,11316
6,reporte_tic_tercer_trimestre_2024_4_1,756075


🧮 Total de registros consolidados: 1,851,254

🔚 Conexión cerrada correctamente.


Bloque para explorar y visualizar la tabla consolidada

Código optimizado para limpiar y renombrar consolidado_tic_4_1

In [ ]:
import duckdb, pandas as pd

# --- Cerrar conexión previa si está abierta ---
try:
    con.close()
    print("🔒 Conexión anterior cerrada correctamente.")
except:
    print("ℹ️ No había conexión previa activa.")

# --- Reconexión a la base ---
db_path = "/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb"
con = duckdb.connect(db_path)
print(f"✅ Conectado a la base: {db_path}")

# --- Leer muestra para detectar encabezado correcto ---
df_preview = con.execute("""
    SELECT * FROM consolidado_tic_4_1
    LIMIT 10
""").fetchdf()

print("\n👀 Vista previa actual:")
display(df_preview)

# --- Extraer nombres reales desde la fila 5 (índice 4) ---
header_row = df_preview.iloc[4, 1:].tolist()
header_row = [h.strip().upper() if isinstance(h, str) else f"COLUMN_{i}" for i, h in enumerate(header_row)]

print("\n🧭 Encabezados detectados:")
print(header_row)

# --- Cargar todo el dataset saltando las filas de metadatos (primeras 5 filas) ---
df = con.execute("""
    SELECT * FROM consolidado_tic_4_1
""").fetchdf().iloc[5:].copy()

# --- Asignar nombres de columnas ---
df.columns = ["origen"] + header_row

# --- Limpiar filas vacías o irrelevantes ---
df = df.dropna(subset=["AÑO", "DEPARTAMENTO", "MUNICIPIO"], how="any")

# --- Convertir tipos numéricos ---
cols_numericas = ["AÑO", "TRIMESTRE", "VELOCIDAD SUBIDA", "VELOCIDAD BAJADA", "No. ACCESOS FIJOS A INTERNET"]
for col in cols_numericas:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# --- Guardar tabla limpia ---
con.register("df_clean", df)
con.execute("CREATE OR REPLACE TABLE consolidado_tic_4_1_limpia AS SELECT * FROM df_clean")
con.unregister("df_clean")

print("\n✅ Tabla 'consolidado_tic_4_1_limpia' creada correctamente.")

# --- Ver estructura y muestra ---
estructura = con.execute("PRAGMA table_info('consolidado_tic_4_1_limpia')").fetchdf()
print("\n📋 Nueva estructura:")
display(estructura)

muestra = con.execute("SELECT * FROM consolidado_tic_4_1_limpia LIMIT 10").fetchdf()
print("\n👀 Vista previa de datos limpios:")
display(muestra)

# --- Cerrar conexión ---
con.close()
print("\n🔚 Conexión cerrada correctamente.")


🔒 Conexión anterior cerrada correctamente.
✅ Conectado a la base: /content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb

👀 Vista previa actual:


,origen,Unnamed: 0,Unnamed: 1,Unnamed: 10,Unnamed: 11,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,articles_404030_archivo_xls_4_1,INFORMACIÓN TRIMESTRAL DE ACCESOS FIJOS A INTE...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,articles_404030_archivo_xls_4_1,2022,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,articles_404030_archivo_xls_4_1,- Cifras remitidas por los Proveedores de Red...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,articles_404030_archivo_xls_4_1,Contenido: Información de accesos fijos a Inte...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,articles_404030_archivo_xls_4_1,AÑO,TRIMESTRE,VELOCIDAD SUBIDA,No. ACCESOS FIJOS A INTERNET,PROVEEDOR,CÓDIGO DANE,DEPARTAMENTO,CÓDIGO DANE,MUNICIPIO,SEGMENTO,TECNOLOGÍA,VELOCIDAD BAJADA
5,articles_404030_archivo_xls_4_1,2022,3,12.5,1,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,19,CAUCA,19698,SANTANDER DE QUILICHAO,CORPORATIVO,WIFI,25
6,articles_404030_archivo_xls_4_1,2022,3,5,1,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,19,CAUCA,19698,SANTANDER DE QUILICHAO,CORPORATIVO,WIFI,10
7,articles_404030_archivo_xls_4_1,2022,3,2.5,49,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,19,CAUCA,19698,SANTANDER DE QUILICHAO,RESIDENCIAL - ESTRATO 2,WIFI,5
8,articles_404030_archivo_xls_4_1,2022,3,4,1,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,19,CAUCA,19698,SANTANDER DE QUILICHAO,CORPORATIVO,WIFI,8
9,articles_404030_archivo_xls_4_1,2022,3,2.5,2,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,19,CAUCA,19698,SANTANDER DE QUILICHAO,CORPORATIVO,WIFI,5



🧭 Encabezados detectados:
['AÑO', 'TRIMESTRE', 'VELOCIDAD SUBIDA', 'NO. ACCESOS FIJOS A INTERNET', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA']


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


✅ Tabla 'consolidado_tic_4_1_limpia' creada correctamente.

📋 Nueva estructura:


,cid,name,type,notnull,dflt_value,pk
0,0,origen,VARCHAR,False,None,False
1,1,AÑO,DOUBLE,False,None,False
2,2,TRIMESTRE,DOUBLE,False,None,False
3,3,VELOCIDAD SUBIDA,DOUBLE,False,None,False
4,4,NO. ACCESOS FIJOS A INTERNET,VARCHAR,False,None,False
5,5,PROVEEDOR,VARCHAR,False,None,False
6,6,CÓDIGO DANE,VARCHAR,False,None,False
7,7,DEPARTAMENTO,VARCHAR,False,None,False
8,8,CÓDIGO DANE_1,VARCHAR,False,None,False
9,9,MUNICIPIO,VARCHAR,False,None,False



👀 Vista previa de datos limpios:


,origen,AÑO,TRIMESTRE,VELOCIDAD SUBIDA,NO. ACCESOS FIJOS A INTERNET,PROVEEDOR,CÓDIGO DANE,DEPARTAMENTO,CÓDIGO DANE_1,MUNICIPIO,SEGMENTO,TECNOLOGÍA,VELOCIDAD BAJADA
0,articles_404030_archivo_xls_4_1,2022.0,3.0,12.5,1,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,19,CAUCA,19698,SANTANDER DE QUILICHAO,CORPORATIVO,WIFI,25.0
1,articles_404030_archivo_xls_4_1,2022.0,3.0,5.0,1,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,19,CAUCA,19698,SANTANDER DE QUILICHAO,CORPORATIVO,WIFI,10.0
2,articles_404030_archivo_xls_4_1,2022.0,3.0,2.5,49,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,19,CAUCA,19698,SANTANDER DE QUILICHAO,RESIDENCIAL - ESTRATO 2,WIFI,5.0
3,articles_404030_archivo_xls_4_1,2022.0,3.0,4.0,1,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,19,CAUCA,19698,SANTANDER DE QUILICHAO,CORPORATIVO,WIFI,8.0
4,articles_404030_archivo_xls_4_1,2022.0,3.0,2.5,2,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,19,CAUCA,19698,SANTANDER DE QUILICHAO,CORPORATIVO,WIFI,5.0
5,articles_404030_archivo_xls_4_1,2022.0,3.0,6.0,2,@DIGITAL GROUP SAS,05,ANTIOQUIA,05031,AMALFI,RESIDENCIAL - ESTRATO 4,FIBER TO THE HOME (FTTH),12.0
6,articles_404030_archivo_xls_4_1,2022.0,3.0,10.0,1,@DIGITAL GROUP SAS,05,ANTIOQUIA,05031,AMALFI,RESIDENCIAL - ESTRATO 4,FIBER TO THE HOME (FTTH),20.0
7,articles_404030_archivo_xls_4_1,2022.0,3.0,10.0,9,@DIGITAL GROUP SAS,05,ANTIOQUIA,05031,AMALFI,RESIDENCIAL - ESTRATO 3,FIBER TO THE HOME (FTTH),20.0
8,articles_404030_archivo_xls_4_1,2022.0,3.0,4.0,6,@DIGITAL GROUP SAS,05,ANTIOQUIA,05031,AMALFI,RESIDENCIAL - ESTRATO 3,FIBER TO THE HOME (FTTH),8.0
9,articles_404030_archivo_xls_4_1,2022.0,3.0,6.0,35,@DIGITAL GROUP SAS,05,ANTIOQUIA,05031,AMALFI,RESIDENCIAL - ESTRATO 1,FIBER TO THE HOME (FTTH),12.0



🔚 Conexión cerrada correctamente.


Validación de cobertura y consistencia

In [ ]:
import duckdb, pandas as pd

db_path = "/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb"
con = duckdb.connect(db_path, read_only=True)

print("✅ Conectado a la base para análisis exploratorio.\n")

# --- 1️⃣ Departamentos únicos y conteo ---
print("📍 Departamentos y número de municipios reportados:")
deptos = con.execute("""
    SELECT DEPARTAMENTO, COUNT(DISTINCT MUNICIPIO) AS municipios_unicos, COUNT(*) AS total_registros
    FROM consolidado_tic_4_1_limpia
    GROUP BY DEPARTAMENTO
    ORDER BY DEPARTAMENTO
""").fetchdf()
display(deptos)

# --- 2️⃣ Cobertura temporal ---
print("\n🗓️ Años y trimestres disponibles:")
periodos = con.execute("""
    SELECT AÑO, TRIMESTRE, COUNT(*) AS registros
    FROM consolidado_tic_4_1_limpia
    GROUP BY AÑO, TRIMESTRE
    ORDER BY AÑO, TRIMESTRE
""").fetchdf()
display(periodos)

# --- 3️⃣ Validación de velocidades ---
print("\n📈 Rango de velocidades detectadas:")
velocidades = con.execute("""
    SELECT
        MIN("VELOCIDAD SUBIDA") AS min_subida,
        MAX("VELOCIDAD SUBIDA") AS max_subida,
        MIN("VELOCIDAD BAJADA") AS min_bajada,
        MAX("VELOCIDAD BAJADA") AS max_bajada,
        ROUND(AVG("VELOCIDAD SUBIDA"),2) AS promedio_subida,
        ROUND(AVG("VELOCIDAD BAJADA"),2) AS promedio_bajada
    FROM consolidado_tic_4_1_limpia
""").fetchdf()
display(velocidades)

con.close()
print("\n🔚 Conexión cerrada correctamente.")


✅ Conectado a la base para análisis exploratorio.

📍 Departamentos y número de municipios reportados:


,DEPARTAMENTO,municipios_unicos,total_registros
0,AMAZONAS,9,1078
1,ANTIOQUIA,125,287503
2,ARAUCA,7,9670
3,"ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANT...",2,2071
4,ATLÁNTICO,23,66524
5,BOGOTÁ D.C.,1,59727
6,BOLÍVAR,46,51040
7,BOYACÁ,123,84454
8,CALDAS,27,67577
9,CAQUETA,16,13258



🗓️ Años y trimestres disponibles:


,AÑO,TRIMESTRE,registros
0,2021.0,4.0,104213
1,2021.0,NaN,1
2,2022.0,1.0,184864
3,2022.0,2.0,370909
4,2022.0,3.0,568156
5,2022.0,4.0,589421
6,2022.0,NaN,2
7,NaN,NaN,12



📈 Rango de velocidades detectadas:


,min_subida,max_subida,min_bajada,max_bajada,promedio_subida,promedio_bajada
0,0.0,3450300.0,0.0,3450300.0,111.03,161.31



🔚 Conexión cerrada correctamente.


In [ ]:
import duckdb
con = duckdb.connect("/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb")
con.execute("SHOW TABLES").fetchdf()


,name
0,articles_404030_archivo_xls_1
1,articles_404030_archivo_xls_10
2,articles_404030_archivo_xls_11
3,articles_404030_archivo_xls_12
4,articles_404030_archivo_xls_13
...,...
96,reporte_tic_tercer_trimestre_2024_7
97,reporte_tic_tercer_trimestre_2024_8
98,reporte_tic_tercer_trimestre_2024_9
99,reporte_tic_tercer_trimestre_2024_CONTENIDO


Verificación de unicidad

In [ ]:
import duckdb
con = duckdb.connect("/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb")

duplicados = con.execute("""
    SELECT COUNT(*) AS total_registros,
           COUNT(DISTINCT (AÑO, TRIMESTRE, DEPARTAMENTO, MUNICIPIO, PROVEEDOR, "TECNOLOGÍA")) AS registros_unicos
    FROM consolidado_tic_4_1_limpia
""").fetchdf()

print(duplicados)
con.close()


   total_registros  registros_unicos
0          1817578             55982


Consulta para verificar dimensiones y muestra de datos

In [ ]:
import duckdb
import pandas as pd

# --- Conexión a la base ---
db_path = "/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb"
con = duckdb.connect(db_path)

# --- 1️⃣ Dimensiones de la tabla ---
dim = con.execute("""
    SELECT
        COUNT(*) AS total_registros,
        COUNT(DISTINCT DEPARTAMENTO) AS departamentos,
        COUNT(DISTINCT MUNICIPIO) AS municipios,
        COUNT(DISTINCT AÑO) AS anios,
        COUNT(DISTINCT TRIMESTRE) AS trimestres,
        COUNT(DISTINCT PROVEEDOR) AS proveedores,
        COUNT(DISTINCT "TECNOLOGÍA") AS tecnologias
    FROM consolidado_tic_4_1_limpia
""").fetchdf()

print("📏 Dimensiones de la tabla `consolidado_tic_4_1_limpia`:")
display(dim)

# --- 2️⃣ Vista previa ordenada por año y trimestre ---
muestra = con.execute("""
    SELECT AÑO, TRIMESTRE, DEPARTAMENTO, MUNICIPIO, PROVEEDOR,
           "SEGMENTO", "TECNOLOGÍA", "VELOCIDAD SUBIDA", "VELOCIDAD BAJADA",
           "NO. ACCESOS FIJOS A INTERNET"
    FROM consolidado_tic_4_1_limpia
    WHERE AÑO IS NOT NULL
    ORDER BY AÑO DESC, TRIMESTRE DESC
    LIMIT 20
""").fetchdf()

print("\n👀 Vista previa de registros representativos:")
display(muestra)

con.close()
print("\n🔚 Conexión cerrada correctamente.")


📏 Dimensiones de la tabla `consolidado_tic_4_1_limpia`:


,total_registros,departamentos,municipios,anios,trimestres,proveedores,tecnologias
0,1817578,35,1037,2,4,1085,18



👀 Vista previa de registros representativos:


,AÑO,TRIMESTRE,DEPARTAMENTO,MUNICIPIO,PROVEEDOR,SEGMENTO,TECNOLOGÍA,VELOCIDAD SUBIDA,VELOCIDAD BAJADA,NO. ACCESOS FIJOS A INTERNET
0,2022.0,4.0,CAUCA,SANTANDER DE QUILICHAO,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,CORPORATIVO,WIFI,30.0,60.0,2
1,2022.0,4.0,CAUCA,SANTANDER DE QUILICHAO,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,CORPORATIVO,WIFI,15.0,30.0,1
2,2022.0,4.0,CAUCA,SANTANDER DE QUILICHAO,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,CORPORATIVO,WIFI,5.0,10.0,3
3,2022.0,4.0,CAUCA,SANTANDER DE QUILICHAO,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,CORPORATIVO,WIFI,4.0,8.0,3
4,2022.0,4.0,CAUCA,SANTANDER DE QUILICHAO,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,CORPORATIVO,WIFI,20.0,40.0,1
5,2022.0,4.0,CAUCA,SANTANDER DE QUILICHAO,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,RESIDENCIAL - ESTRATO 1,WIFI,2.5,5.0,147
6,2022.0,4.0,CAUCA,SANTANDER DE QUILICHAO,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,CORPORATIVO,WIFI,2.5,5.0,3
7,2022.0,4.0,CAUCA,SANTANDER DE QUILICHAO,SERVICIOS DE COMUNICACIONES INTEGRALES S.A.S,CORPORATIVO,WIFI,12.5,25.0,3
8,2022.0,4.0,ANTIOQUIA,AMALFI,@DIGITAL GROUP SAS,RESIDENCIAL - ESTRATO 4,FIBER TO THE HOME (FTTH),7.0,15.0,1
9,2022.0,4.0,ANTIOQUIA,AMALFI,@DIGITAL GROUP SAS,RESIDENCIAL - ESTRATO 4,FIBER TO THE HOME (FTTH),6.0,12.0,5



🔚 Conexión cerrada correctamente.


Bloque de detección y limpieza de outliers

In [ ]:
import duckdb
import pandas as pd
import numpy as np

db_path = "/content/gdrive/MyDrive/trabajoGrado/colombiatic_datos/colombiatic.duckdb"
con = duckdb.connect(db_path)

# --- Cargar los datos a memoria para depuración ---
df = con.execute("SELECT * FROM consolidado_tic_4_1_limpia").fetchdf()

print(f"📊 Registros iniciales: {len(df):,}")

# --- Convertir a numérico por seguridad ---
for col in ["VELOCIDAD SUBIDA", "VELOCIDAD BAJADA"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# --- Calcular IQR para cada variable ---
def limpiar_outliers_iqr(serie):
    q1 = serie.quantile(0.25)
    q3 = serie.quantile(0.75)
    iqr = q3 - q1
    limite_inferior = q1 - 1.5 * iqr
    limite_superior = q3 + 1.5 * iqr
    return serie.clip(lower=limite_inferior, upper=limite_superior)

df["VELOCIDAD SUBIDA_LIMPIA"] = limpiar_outliers_iqr(df["VELOCIDAD SUBIDA"])
df["VELOCIDAD BAJADA_LIMPIA"] = limpiar_outliers_iqr(df["VELOCIDAD BAJADA"])

# --- Conteo de valores corregidos ---
cambios_subida = (df["VELOCIDAD SUBIDA"] != df["VELOCIDAD SUBIDA_LIMPIA"]).sum()
cambios_bajada = (df["VELOCIDAD BAJADA"] != df["VELOCIDAD BAJADA_LIMPIA"]).sum()

print(f"⚙️ Registros ajustados (subida): {cambios_subida:,}")
print(f"⚙️ Registros ajustados (bajada): {cambios_bajada:,}")

# --- Guardar tabla limpia ---
con.execute("DROP TABLE IF EXISTS consolidado_tic_4_1_filtrado")
con.register("df_temp", df)
con.execute("""
    CREATE TABLE consolidado_tic_4_1_filtrado AS
    SELECT
        origen, AÑO, TRIMESTRE, DEPARTAMENTO, MUNICIPIO, PROVEEDOR,
        SEGMENTO, TECNOLOGÍA,
        "NO. ACCESOS FIJOS A INTERNET",
        "CÓDIGO DANE", "CÓDIGO DANE_1",
        "VELOCIDAD SUBIDA_LIMPIA" AS VELOCIDAD_SUBIDA,
        "VELOCIDAD BAJADA_LIMPIA" AS VELOCIDAD_BAJADA
    FROM df_temp
""")
con.unregister("df_temp")

# --- Verificar dimensiones ---
dim = con.execute("SELECT COUNT(*) AS total, COUNT(DISTINCT MUNICIPIO) AS municipios FROM consolidado_tic_4_1_filtrado").fetchdf()
print("\n✅ Tabla `consolidado_tic_4_1_filtrado` creada correctamente:")
display(dim)

con.close()
print("\n🔚 Conexión cerrada correctamente.")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

📊 Registros iniciales: 1,817,578
⚙️ Registros ajustados (subida): 304,034
⚙️ Registros ajustados (bajada): 165,909


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


✅ Tabla `consolidado_tic_4_1_filtrado` creada correctamente:


,total,municipios
0,1817578,1037



🔚 Conexión cerrada correctamente.
